# TPOT in Python

In this tutorial, you will learn how to use a very unique library in python: `TPOT`. The reason why this library is unique is that it automates the entire Machine Learning pipeline and provides you with the best performing machine learning model. `TPOT` is one of the most popular package in Automated Machine Learning (AutoML) - since it is effective and free to use.

More specifically, you will learn:

- The idea behind Automated Machine Learning
- How TPOT uses Genetic Programming to select the best machine learning model
- Using TPOT on a dataset in Python
- Limitations of TPOT

Let's get started!

Original post is [here](https://www.datacamp.com/community/tutorials/tpot-machine-learning-python) - many thanks to DataCamp!

Revised by Dr. Jie Tao, Mar-31-2020, Ver. 1.0

#Introduction

There are a lot of components you have to consider before solving a machine learning problem some of which includes *data preparation*, *feature selection*, *feature engineering*, *model selection and validation*, *hyperparameter tuning*, etc. 

In theory, you can find and apply a plethora of techniques for each of these components, but they all might perform differently for different datasets. The challenge is to find the best performing combination of techniques so that you can minimize the error in your predictions. This is the main reason that nowadays people are working to develop **Auto-ML algorithms** and platforms so that anyone, without any machine learning expertise, can build models without spending much time or effort. 

One such platform is available as a python library: **TPOT**. You can consider **TPOT** your Data Science Assistant. TPOT is a python Automated Machine Learning tool that optimizes *machine learning pipelines* using genetic programming. It will automate the most tedious part of machine learning by intelligently exploring thousands of possible pipelines to find the best one for your data.

Following image depicts how TPOT works:

<img src = 'https://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1537396029/output_2_0_d7uh0v.png'/>

# Installation
To install tpot on your system, you can run the command **sudo pip install tpot** on command line terminal or check out this [link](http://epistasislab.github.io/tpot/installing/). tpot is built on top of several existing Python libraries, including **numpy**, **scipy**, **scikit-learn**, **DEAP**, **update_checker**, **tqdm**, **stopit**, and **pandas**. Most of the necessary Python packages can be installed via the [Anaconda Python distribution](https://www.continuum.io/downloads), or you could install them separately also. Optionally, you can also install **XGBoost** [here](https://www.datacamp.com/community/tutorials/xgboost-in-python) if you would like tpot to use the eXtreme Gradient Boosting models.

# Genetic Programming
With the right data, computing power and machine learning model you can discover a solution to any problem, but knowing which model to use can be challenging for you as there are so many of them like *Decision Trees*, *SVM*, *KNN*, etc. That's where genetic programming can be of great use and provide help. Genetic algorithms are inspired by the Darwinian process of Natural Selection, and they are used to generate solutions to optimization and search problems in computer science.

Broadly speaking, Genetic Algorithms have three properties:

- Selection: You have a population of possible solutions to a given problem and a fitness function. At every iteration, you evaluate how to fit each solution with your fitness function.
- Crossover: Then you select the fittest ones and perform crossover to create a new population.
- Mutation: You take those children and mutate them with some random modification and repeat the process until you get the fittest or best solution.

Genetic Programming itself is a big topic in computer science but if you wish to go deeper into genetic algorithms, check out this video series.

***But how does this all fit into data science?***

Well, it turns out that choosing the right machine learning model and all the best hyperparameters for that model is itself an optimization problem for which genetic programming can be used. The Python library tpot built on top of scikit-learn uses genetic programming to optimize your machine learning pipeline. For instance, in machine learning, after preparing your data you need to know what features to input to your model and how you should construct those features. Once you have those features, you input them into your model to train on, and then you tune your hyperparameters to get the optimal results. Instead of doing this all by yourselves through trial and error, TPOT automates these steps for you with genetic programming and outputs the optimal code for you when it's done.

# A Running Example


To understand further, you will practice using the tpot library on an open source dataset. The dataset you will be using is MAGIC Gamma Telescope DataSet. The data are generated to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. The dataset has the following attributes/features :

**Predictors**

1. fLength: continuous *# major axis of ellipse [mm]*
2. fWidth: continuous *# minor axis of ellipse [mm]*
3. fSize: continuous *# 10-log of sum of content of all pixels [in #phot]*
4. fConc: continuous *# ratio of sum of two highest pixels over fSize [ratio]*
5. fConc1: continuous *# ratio of highest pixel over fSize [ratio]*
6. fAsym: continuous *# distance from the highest pixel to center, projected onto major axis [mm]*
7. fM3Long: continuous *# 3rd root of the third moment along major axis [mm]*
8. fM3Trans: continuous *# 3rd root of the third moment along minor axis [mm]*
9. fAlpha: continuous *# angle of major axis with a vector to origin [deg]*
10. fDist: continuous *# distance from the origin to the center of the ellipse [mm]*

**Target**

11. class: g,h *# gamma (signal), hadron (background) (target variable)*

The goal is to classify an observation as *gamma*, which is the desired signal, or *hadron*, which is the background noise, based on the attributes provided.

You will start by importing **pandas** and **numpy** libraries to read the data and perform computations on it respectively.

In [0]:
import pandas as pd
import numpy as np

## Read-in Data

Use the pandas **read_csv()** function to read the dataset as a DataFrame. Also, specify the parameter **header = None** since the dataset doesn't have column names yet.

In [0]:
data_url = 'https://raw.githubusercontent.com/fairfield-university-ba545/Week11/master/data/magic04.csv'
telescope_data=pd.read_csv(data_url,header=None)

Check the contents of the DataFrame using the **head()** method.

In [0]:
telescope_data.head()

,0,1,2,3,4,5,6,7,8,9,10
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


To give names to the columns of the DataFrame, you can use the attribute columns on the pandas DataFrame and assign it to a list containing the names of the columns.

In [0]:
telescope_data.columns = ['fLength', 'fWidth','fSize','fConc','fConcl','fAsym','fM3Long','fM3Trans','fAlpha','fDist','class']

In [0]:
telescope_data.head()

,fLength,fWidth,fSize,fConc,fConcl,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


Now you have the column names as well in your DataFrame.

# Investigating Your Data
To get the necessary information about the DataFrame like the number of values in each column, data-type, count, mean, etc., you can use the pandas info() and describe() methods.

In [0]:
telescope_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19020 entries, 0 to 19019
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   fLength   19020 non-null  float64
 1   fWidth    19020 non-null  float64
 2   fSize     19020 non-null  float64
 3   fConc     19020 non-null  float64
 4   fConcl    19020 non-null  float64
 5   fAsym     19020 non-null  float64
 6   fM3Long   19020 non-null  float64
 7   fM3Trans  19020 non-null  float64
 8   fAlpha    19020 non-null  float64
 9   fDist     19020 non-null  float64
 10  class     19020 non-null  object 
dtypes: float64(10), object(1)
memory usage: 1.6+ MB


In [0]:
telescope_data.describe()

,fLength,fWidth,fSize,fConc,fConcl,fAsym,fM3Long,fM3Trans,fAlpha,fDist
count,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000,19020.000000
mean,53.250154,22.180966,2.825017,0.380327,0.214657,-4.331745,10.545545,0.249726,27.645707,193.818026
std,42.364855,18.346056,0.472599,0.182813,0.110511,59.206062,51.000118,20.827439,26.103621,74.731787
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863800,2.477100,0.235800,0.128475,-20.586550,-12.842775,-10.849375,5.547925,142.492250
50%,37.147700,17.139900,2.739600,0.354150,0.196500,4.013050,15.314100,0.666200,17.679500,191.851450
75%,70.122175,24.739475,3.101600,0.503700,0.285225,24.063700,35.837800,10.946425,45.883550,240.563825
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


Turns out all the features in the DataFrame are continuous in nature. The target variable/feature class is however *categorical*. You can check the counts of each class in the target variable using **value_counts()** method.


In [0]:
telescope_data['class'].value_counts()

g    12332
h     6688
Name: class, dtype: int64

It's generally a good idea to randomly **shuffle** the data before starting to avoid any type of ordering in the data. You can rearrange the data in the DataFrame using numpy's **random** and **permutation()** function. To reset the index numbers after the shuffle use **reset_index()** method with **drop = True** as a parameter.

In [0]:
telescope_shuffle=telescope_data.iloc[np.random.permutation(len(telescope_data))]
tele=telescope_shuffle.reset_index(drop=True)
tele.head()

,fLength,fWidth,fSize,fConc,fConcl,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,87.2570,23.2826,3.0183,0.2167,0.1146,-31.4565,68.7719,-4.4924,2.3986,297.9090,g
1,28.4792,24.3384,2.4031,0.6522,0.3379,-30.0060,-21.9428,18.6641,11.0181,314.8710,g
2,38.2371,19.4698,2.9393,0.2691,0.1593,-18.0537,-17.9550,-8.4689,8.6172,140.5940,g
3,81.6368,25.1357,2.4476,0.4251,0.2132,-96.2788,37.9453,7.9119,18.7284,328.0580,h
4,205.1280,176.3350,4.9946,0.0139,0.0073,8.5005,-79.1621,83.2966,76.3946,41.4222,g


### Label Your Target

Before using tpot, it is essential you do the labeling of categorical variables in your DataFrame. To learn more about labeling categorical variables check out this tutorial. Here only the target variable class is the *categorical* feature, hence the treatment should be done to that column. 

You will replace the **g** class (signal) with a numerical value **0** and **h** class (background) with numerical value **1**. This can be achieved via **map()** function with the argument as a dictionary which holds the respective mapping of the classes of the target variable.

**NOTE:** in practice, this is usually done by the [label encoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) or [one-hot encoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) provided with scikit-learn.

In [0]:
tele['class']=tele['class'].map({'g':0,'h':1})
tele.head()

,fLength,fWidth,fSize,fConc,fConcl,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,87.2570,23.2826,3.0183,0.2167,0.1146,-31.4565,68.7719,-4.4924,2.3986,297.9090,0
1,28.4792,24.3384,2.4031,0.6522,0.3379,-30.0060,-21.9428,18.6641,11.0181,314.8710,0
2,38.2371,19.4698,2.9393,0.2691,0.1593,-18.0537,-17.9550,-8.4689,8.6172,140.5940,0
3,81.6368,25.1357,2.4476,0.4251,0.2132,-96.2788,37.9453,7.9119,18.7284,328.0580,1
4,205.1280,176.3350,4.9946,0.0139,0.0073,8.5005,-79.1621,83.2966,76.3946,41.4222,0


Now you store the class labels, which you need to predict, in a separate variable *tele_class*.

In [0]:
tele_class = tele['class'].values

### MIssing Value Handling

You should also do missing value treatment before using *tpot*. To check the number of missing values column-wise, you can execute the following:

In [0]:
pd.isnull(tele).any()

fLength     False
fWidth      False
fSize       False
fConc       False
fConcl      False
fAsym       False
fM3Long     False
fM3Trans    False
fAlpha      False
fDist       False
class       False
dtype: bool

This dataset doesn't have any missing values. Note in datasets with missing values you can either drop the rows/columns using dropna() method or replace the missing value with some dummy value using fillna() method. For example, the following chunk of code will replace the NA values with a dummy value -999.

tele = tele.fillna(-999)

You will now split the DataFrame into a training set and a testing set just like you do while doing any type of machine learning modeling. You can do this via sklearn's **cross_validation** **train_test_split**. The parameters are tele.index as indexes of the DataFrame, *train_size = 0.75* to keep 75% of the data in training set, *test_size = 0.25* to keep the rest 25% data in testing set and stratify = tele_class the class label's values in the dataset. Note the validation set is just to give us an idea of the test set error. Here it is kept to be the same as a test set.

In [0]:
from sklearn.model_selection import train_test_split
training_indices, testing_indices = train_test_split(tele.index,
                                                        stratify = tele_class,
                                                        train_size=0.75, test_size=0.25, random_state = 2019)


You can check the size of the training set and validation set using the size attribute.

In [0]:
training_indices.size, testing_indices.size

(14265, 4755)

Now it's time to use the **tpot** library to suggest us the best pipeline for this binary classification problem. To do so, you have to import **TPOTClassifier** class from the tpot library. Had this been a regression problem you would have imported **TPOTRegressor** class.

**TPOTClassifier** has a wide variety of parameters, and you can read all about them here. But the most notable ones you must know are:

- generations: Number of iterations to the run pipeline optimization process. The default is 100.
- population_size: Number of individuals to retain in the genetic programming population every generation. The default is 100.
- offspring_size: Number of offspring to produce in each genetic programming generation. The default is 100.
- mutation_rate: Mutation rate for the genetic programming algorithm in the range [0.0, 1.0]. This parameter tells the GP algorithm how many pipelines to apply random changes to every generation. Default is 0.9
- crossover_rate: Crossover rate for the genetic programming algorithm in the range [0.0, 1.0]. This parameter tells the genetic programming algorithm how many pipelines to "breed" every generation.
- scoring: Function used to evaluate the quality of a given pipeline for the classification problem like accuracy, average_precision, roc_auc, recall, etc. The default is accuracy.
- cv: Cross-validation strategy used when evaluating pipelines. The default is 5.
- random_state: The seed of the pseudo-random number generator used in TPOT. Use this parameter to make sure that TPOT will give you the same results each time you run it against the same data set with that seed.

Also note mutation_rate + crossover_rate cannot exceed **1.0**.

Here you will use tpot with generations = 5 and the rest of the parameters at default values. The parameter verbosity = 2 states how much information TPOT communicates while it's running.

Then you will call the **fit()** method with the training set (without the target column) and the target column as the arguments.

Note running the code in the below cell will take several hours to finish. With the given TPOT settings (5 generations with 100 population size), TPOT will evaluate 500 pipeline configurations before finishing. To put this number into context, think about a grid search of 500 hyperparameter combinations for a machine learning algorithm and how long that grid search will take. That is 500 model configurations to evaluate with 5-fold cross-validation, which means that roughly 2500 models are fit and evaluated on the training data in one grid search. That's a time-consuming procedure! Later, you will get to know about some more arguments that you can pass to TPOTClassifier to control the execution time for TPOT to finish.

In [0]:
#### install TPOT
# !pip install tpot

In [0]:
from tpot import TPOTClassifier
# from tpot import TPOTRegressor # for regression tasks

tpot = TPOTClassifier(generations=5,verbosity=2)

tpot.fit(tele.drop('class',axis=1).loc[training_indices].values, # X_train
         tele.loc[training_indices,'class'].values) # y_train

Generation 1 - Current best internal CV score: 0.8737469330529268
Generation 2 - Current best internal CV score: 0.8790746582544691
Generation 3 - Current best internal CV score: 0.880266386260077
Generation 4 - Current best internal CV score: 0.880266386260077
Generation 5 - Current best internal CV score: 0.880266386260077

Best pipeline: RandomForestClassifier(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), bootstrap=True, criterion=gini, max_features=0.45, min_samples_leaf=2, min_samples_split=13, n_estimators=100)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=100,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In the above, 5 generations were computed, each giving the training efficiency of the fitting model on the training set. As evident, the best pipeline is the one that has the CV accuracy score of **88.12%**. The model that produces this result is the pipeline, consisting of pre-processing techniques like `PloynomialFeatures` and then `RobustScaler` that adds synthetic features to the input data and normalizes them, which then get utilized by a Gradient Boosting classifier to form the final predictions. You can also notice that tpot gives various hyper-parameter values like `learning_rate`, `max_depth`, etc., also along with the classifier.

One of the key difference here is we use both `X_test` and `y_test` in the code below, since the `.score()` method below combines the __prediction__ and __evaluation__ in the same step.

In [0]:
tpot.score(tele.drop('class',axis=1).loc[testing_indices].values, #X_test
           tele.loc[testing_indices, 'class'].values) # y_test

0.88012618296529965

As can be seen, the test accuracy is **88.14%.**

Finally, you can tell TPOT to export the corresponding Python code for the optimized pipeline to a text file with the export function:

tpot.export('tpot_MAGIC_Gamma_Telescope_pipeline.py')

Isn't that awesome? Without you tweaking a lot of parameters and options to get the best model, TPOT not only gave you the information about the best model but also a working code for it!

As indicated earlier, the last TPOT run took *several hours* to finish. Well, there are certain parameters you can specify to control the execution time of TPOT but with a trade-off. Since you will be limiting the time of TPOT execution, TPOT won't be able to explore all the possible pipelines and hence the best model suggested by TPOT at the end of the constrained time limit may not be the best model possible for that dataset. 

However, if sufficient time is given it will be somewhat closer to the best possible model. Some parameters are:

- **max_time_mins**: how many minutes TPOT has to optimize the pipeline. If not None, this setting will override the generations parameter and allow TPOT to run until max_time_mins minutes elapse.
- **max_eval_time_mins**: how many minutes TPOT has to evaluate a single pipeline. Setting this parameter to higher values will enable TPOT to evaluate more complex pipelines, but will also allow TPOT to run longer. Use this parameter to help prevent TPOT from wasting time on assessing time-consuming pipelines. The default is 5.
- **early_stop**: how many generations TPOT checks whether there is no improvement in the optimization process. Ends the optimization process if there is no improvement in the given number of generations.
- **n_jobs**: Number of procedures to use in parallel for evaluating pipelines during the TPOT optimization process. Setting n_jobs=-1 will use as many cores as available on the computer. Beware that using multiple methods on the same machine may cause memory issues for large datasets. The default is 1.
- **subsample**: Fraction of training samples that are used during the TPOT optimization process. Must be in the range (0.0, 1.0]. The default is 1.

Just for practice, you will again run TPOT with additional arguments max_time_mins = 10 and max_eval_time_mins = 0.04 but this time with reduced population_size = 15. If you do not want to wait hours for your TPOT model, this might be the way to go!

In [0]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=10, max_eval_time_mins=0.04, population_size=15)
tpot.fit(tele.drop('class',axis=1).loc[training_indices].values, # X_train
         tele.loc[training_indices,'class'].values) # y_train

Generation 1 - Current best internal CV score: 0.8281811731066163
Generation 2 - Current best internal CV score: 0.8467575945563388
Generation 3 - Current best internal CV score: 0.8654739243774452
Generation 4 - Current best internal CV score: 0.8654739243774452
Generation 5 - Current best internal CV score: 0.8654739243774452
Generation 6 - Current best internal CV score: 0.8729749730857256
Generation 7 - Current best internal CV score: 0.8729749730857256

2.009534283333333 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: XGBClassifier(input_matrix, learning_rate=0.5, max_depth=2, min_child_weight=15, n_estimators=100, nthread=1, subsample=0.9)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
        disable_update_check=False, early_stop=None, generations=1000000,
        max_eval_time_mins=0.04, max_time_mins=2, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=15,
        random_state=None, scoring=None, subsample=1.0, use_dask=False,
        verbosity=2, warm_start=False)

As you can notice the best performing classifier within the time frame specified is now XGBoost with `SelectPercentile()` and `RobustScaler()` as the pre-processing steps. But you notice that the accurracy declined to 87.29%.

After you trained your best model, you can always export the pipeline as a file and use it without any training (we know training takes a lot of time).

You can export the above trained pipeline as (assume you have a `model` subfolder in your repo):
```python
tpot.export('tpot_exported_pipeline.py')
```

Then in your subsequent analysis, you can import this `.py` file and then use the `tpot.score()` method to evaluate/deploy the model on _new, unseen_ data.

For more examples of using TPOT for machine learning, refer to [these examples](https://epistasislab.github.io/tpot/examples/).

# Limitations

- **TPOT can take a long time to finish its search**

Running TPOT isn’t as simple as fitting one model on the dataset. It is considering multiple machine learning algorithms (random forests, linear models, SVMs, etc.) in a pipeline with numerous preprocessing steps (missing value imputation, scaling, PCA, feature selection, etc.), the hyper-parameters for all of the models and preprocessing steps, as well as multiple ways to ensemble or stack the algorithms within the pipeline. That’s why it usually takes a long time to execute and isn’t feasible for large datasets.

- **TPOT can recommend different solutions for the same dataset**

If you're working with a reasonably complex dataset or run TPOT for a short amount of time, different TPOT runs may result in different pipeline recommendations. When two TPOT runs recommend different pipelines, this means that the TPOT runs didn't converge due to lack of time or that multiple pipelines perform more-or-less the same on your dataset. If that is the case, you can always use the __voting__ mechanism in the __ensemble learning__ to combine the models together.

# Conclusion

Hurray! You have come to the end of this tutorial. You started with a little introduction about the idea behind Automated Machine Learning platforms. Then you explored a bit on Genetic Programming and how TPOT uses it to automate the machine learning pipeline building process. You also practiced using the TPOT library in Python on a dataset along with gaining knowledge about the functions it provides. If you plan to use TPOT in the future, I strongly suggest you look at its excellent [documentation](http://epistasislab.github.io/tpot/). Happy Exploring!

If you would like to learn more about Machine Learning in Python, take DataCamp's [Machine Learning with Tree-Based Models in Python](https://www.datacamp.com/courses/machine-learning-with-tree-based-models-in-python) course.

The following web pages were used as sources in writing this tutorial.

- [Classification](http://epistasislab.github.io/tpot/api/)
- [What to expect from AutoML software](https://epistasislab.github.io/tpot/using/)